In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [ ]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

/content/drive/MyDrive/Dacon/LDI


In [ ]:
train_err_data = pd.read_csv('./dataset/train_err_data.csv',thousands = ',')
train_quality_data = pd.read_csv('./dataset/train_quality_data.csv',thousands = ',')
train_problem_data = pd.read_csv('./dataset/train_problem_data.csv',thousands = ',')

In [ ]:
df_tqd = train_quality_data.drop_duplicates()
df_ted = train_err_data.drop_duplicates()
df_tpd = train_problem_data.drop_duplicates()

In [ ]:
df_tqd.columns

Index(['time', 'user_id', 'quality_0', 'quality_1', 'quality_2', 'quality_3',
       'quality_4', 'quality_5', 'quality_6', 'quality_7', 'quality_8',
       'quality_9', 'quality_10', 'quality_11', 'quality_12'],
      dtype='object')

In [ ]:
df_ted.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'connection timeout'),'errtype'] = '23_ct'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'active'),'errtype'] = '23_as'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'standby'),'errtype'] = '23_ac'
df_ted.loc[(df_ted['errtype'] == 31) & (df_ted['errcode'] == '0'),'errtype'] = '31_0'
df_ted.loc[(df_ted['errtype'] == 40) & (df_ted['errcode'] == '0'),'errtype'] = '40_0'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
for i in range(0,13):
    qstr = 'quality_'+str(i)
    df_tqd.loc[df_tqd[qstr] != -1,qstr] = 0
    df_tqd.loc[df_tqd[qstr] == -1,qstr] = 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
df_ted = df_ted.drop('fwver',axis=1)
df_ted = df_ted.drop('model_nm',axis=1)
df_tqd = df_tqd.drop('fwver',axis=1)
df_ted = df_ted.drop('errcode',axis=1)

In [ ]:
df_encoded_ted = pd.get_dummies(df_ted,columns=['errtype'])

In [ ]:
times = list(df_ted['time'])

In [ ]:
times = list(df_ted['time'])

for i in range(len(times)):
    try:
        times[i] = int(str(times[i])[4:8])
    except ValueError:
        print(i)
    

df_ted['time'] = times

df_ted.head()

,user_id,time,errtype
0,10000,1101,15
1,10000,1101,12
2,10000,1101,11
3,10000,1101,16
4,10000,1101,4


In [ ]:
print(len(df_ted['time'].unique()))
print(len(df_ted['errtype'].unique()))

33
46


In [ ]:
df_err = df_ted[['user_id','errtype']].values
df_err

array([[10000, 15],
       [10000, 12],
       [10000, 11],
       ...,
       [24999, 4],
       [24999, 4],
       [24999, 15]], dtype=object)

In [ ]:
from tqdm import tqdm

df_x = np.zeros((15000,60))
df_err = df_ted[['user_id','errtype']].values
for pi,err in tqdm(df_err):
    if err == '23_ct':
        err = 43
    elif err == '23_as':
        err = 44
    elif err == '23_ac':
        err = 45
    elif err == '31_0':
        err = 46
    elif err == '40_0':
        err = 47
    df_x[int(pi)-10000,int(err)-1] += 1

for i in range(0,13):
    a = "quality_"+ str(i)
    df_qua = df_tqd[['user_id',a]].values
    for pi, qua in tqdm(df_qua):
        df_x[int(pi)-10000,i + 47] += 1

df_err.shape

100%|██████████| 284193/284193 [00:00<00:00, 379798.48it/s]


(15368002, 2)

In [ ]:
df_x.shape

(15000, 60)

In [54]:
df_x[:10]

array([[  0.,   0.,   8., 104.,   0.,   1.,   1.,   0.,   0.,   7.,  15.,
         16.,   1.,  10.,  59.,  60.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,  32.,   1.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   4.,   4.,   4.,   4.,   4.,   4.,   4.,   4.,
          4.,   4.,   4.,   4.,   4.],
       [  0.,   0.,   0.,   0.,  48.,   1.,   1.,   0.,   0.,   0.,  10.,
         11.,   1.,   8., 143., 128.,   0.,   4.,   1.,   3.,   1., 756.,
          0.,   5.,   1.,  22.,   0.,   0.,   0.,   0., 124.,   0.,  10.,
         18.,   0.,   1.,   1.,   0.,   0.,  57.,  56.,   1.,   0., 126.,
        625., 126.,  56.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   2., 131.,   1.,   2.,   1.,   0.,   0.,   1.,  13.,
         14.,   1.,   4.,  52.,  52.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   2.,   0.,  25.,   

In [ ]:
import pickle
with open('list12.txt', 'wb') as f:
    pickle.dump(df_x, f)